In [ ]:
import numpy as np
import qecstruct as qec
import matplotlib.pyplot as plt
from mdopt.optimiser.utils import (
    ConstraintString,
    IDENTITY,
    SWAP,
    XOR_BULK,
    XOR_LEFT,
    XOR_RIGHT,
)
from examples.decoding.decoding import (
    linear_code_constraint_sites,
    linear_code_prepare_message,
    linear_code_codewords,
)
from examples.decoding.decoding import (
    apply_bitflip_bias,
    apply_constraints,
    decode_linear,
)
from mdopt.mps.utils import (
    create_simple_product_state,
    create_custom_product_state,
)
from mdopt.utils.utils import mpo_to_matrix

In [ ]:
failures_statistics = {}
failures = []

SEED = 1964
NUM_BITS = 16
PROB_ERROR = 0.1
CHECK_DEGREE, BIT_DEGREE = 4, 3
NUM_CHECKS = int(BIT_DEGREE * NUM_BITS / CHECK_DEGREE)
if NUM_BITS / NUM_CHECKS != CHECK_DEGREE / BIT_DEGREE:
    raise ValueError("The Tanner graph of the code must be bipartite.")

PROB_BIAS = PROB_ERROR

CHI_MAX_CONTRACTOR = 128
CHI_MAX_DMRG = 128
CUT = 1e-12
NUM_RUNS = 1

code = qec.random_regular_code(
    NUM_BITS, NUM_CHECKS, BIT_DEGREE, CHECK_DEGREE, qec.Rng(SEED)
)
code_constraint_sites = linear_code_constraint_sites(code)
INITIAL_CODEWORD, PERTURBED_CODEWORD = linear_code_prepare_message(
    code, PROB_ERROR, error_model=qec.BinarySymmetricChannel, seed=SEED
)
tensors = [XOR_LEFT, XOR_BULK, SWAP, XOR_RIGHT]

initial_codeword_state = create_custom_product_state(
    INITIAL_CODEWORD, form="Right-canonical"
)
perturbed_codeword_state = create_custom_product_state(
    PERTURBED_CODEWORD, form="Right-canonical"
)

perturbed_codeword_state = apply_bitflip_bias(
    mps=perturbed_codeword_state,
    sites_to_bias="All",
    prob_bias_list=PROB_BIAS,
    renormalise=True,
    result_to_explicit=False,
)

In [ ]:
# bond dims and entropy vs time
# check codewords

In [ ]:
perturbed_codeword_state = apply_constraints(
    perturbed_codeword_state,
    code_constraint_sites,
    tensors,
    chi_max=CHI_MAX_CONTRACTOR,
    renormalise=True,
    result_to_explicit=False,
    strategy="Naive",
    silent=True,
)